<a href="https://colab.research.google.com/github/faizankshaikh/ForaGym/blob/main/examples/trial1_foragymSimple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone --quiet https://github.com/faizankshaikh/ForaGym.git
%cd ForaGym
!pip install -q -e .
!pip install stable-baselines3

In [4]:
import gym
import foragym
import numpy as np

from itertools import product
from stable_baselines3 import DQN
from stable_baselines3.common.evaluation import evaluate_policy

In [5]:
env = gym.make("foragym:foragym/ForaGym-v0")

c:\Users\faiza\anaconda3\lib\site-packages\gym\core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
c:\Users\faiza\anaconda3\lib\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [3]:
for key, items in env.P.items():
    days_left, life_point, field, weather = env.decode(key)
    p_wait, new_state_wait, reward_wait, is_dead_wait = items[0][0]
    p_forage_failure, new_state_forage_failure, reward_forage_failure, is_dead_forage_failure = items[1][0]

    print("="*10)
    print(f"days_left: {days_left}, life_point: {life_point}, field: {field}, weather: {weather}, enc_state: {key}")
    print(f"p_wait: {p_wait:.2f}, new_state_wait: {new_state_wait}, reward_wait: {reward_wait}, is_dead_wait: {is_dead_wait}")
    print(f"p_forage_failure: {p_forage_failure:.2f}, new_state_forage_failure: {new_state_forage_failure}, reward_forage_failure: {reward_forage_failure}, is_dead_forage_failure: {is_dead_forage_failure}")
    try:
        p_forage_success, new_state_forage_success, reward_forage_success, is_dead_forage_success = items[1][1]
        print(f"p_forage_success: {p_forage_success:.2f}, new_state_forage_success: {new_state_forage_success}, reward_forage_success: {reward_forage_success}, is_dead_forage_success: {is_dead_forage_success}")
    except:
        pass
    print("="*10)
        

days_left: 0, life_point: 0, field: 0, weather: 0, enc_state: 0
p_wait: 1.00, new_state_wait: 0, reward_wait: -1, is_dead_wait: True
p_forage_failure: 1.00, new_state_forage_failure: 0, reward_forage_failure: -1, is_dead_forage_failure: True
days_left: 0, life_point: 0, field: 0, weather: 1, enc_state: 1
p_wait: 1.00, new_state_wait: 0, reward_wait: -1, is_dead_wait: True
p_forage_failure: 1.00, new_state_forage_failure: 0, reward_forage_failure: -1, is_dead_forage_failure: True
days_left: 0, life_point: 0, field: 1, weather: 0, enc_state: 2
p_wait: 1.00, new_state_wait: 0, reward_wait: -1, is_dead_wait: True
p_forage_failure: 1.00, new_state_forage_failure: 0, reward_forage_failure: -1, is_dead_forage_failure: True
days_left: 0, life_point: 0, field: 1, weather: 1, enc_state: 3
p_wait: 1.00, new_state_wait: 0, reward_wait: -1, is_dead_wait: True
p_forage_failure: 1.00, new_state_forage_failure: 0, reward_forage_failure: -1, is_dead_forage_failure: True
days_left: 0, life_point: 0, fie

In [4]:
num_episodes = 2

for episode in range(num_episodes):
    print(f"Episode #{episode+1}")
    print("=" * 10)

    is_dead = False
    obs = env.reset()

    print("Initial state:-")
    env.render()
    print()

    while not is_dead:
        action = 0 #env.action_space.sample()
        print(f"Action to take: {env.ACTION_DICT[action]}")
        state, reward, is_dead, info = env.step(action)
        print(f'Chance to find food: {info["chance"]:.2f}')
        print(f"Reward?: {reward}")
        print(f"is Dead?: {is_dead}")
        print()
        print("Current game state:-")
        env.render()
        print()

    print("-" * 10)

Episode #1
Initial state:-
--Days left: 6
--State of Field: [1 1 0 1 0]
--Current life: 5
--Type of Weather: Clear

Action to take: Wait
Chance to find food: 0.00
Reward?: 0
is Dead?: False

Current game state:-
--Days left: 5
--State of Field: [0 0 0 0 1]
--Current life: 4
--Type of Weather: Clear

Action to take: Wait
Chance to find food: 0.00
Reward?: 0
is Dead?: False

Current game state:-
--Days left: 4
--State of Field: [1 0 1 0 0]
--Current life: 3
--Type of Weather: Rainy

Action to take: Wait
Chance to find food: 0.00
Reward?: 0
is Dead?: False

Current game state:-
--Days left: 3
--State of Field: [1 0 0 1 1]
--Current life: 2
--Type of Weather: Rainy

Action to take: Wait
Chance to find food: 0.00
Reward?: 0
is Dead?: False

Current game state:-
--Days left: 2
--State of Field: [0 0 1 1 1]
--Current life: 1
--Type of Weather: Clear

Action to take: Wait
Chance to find food: 0.00
Reward?: -1
is Dead?: True

Current game state:-
--Days left: 0
--State of Field: [0 1 1 1 1]
--C

In [11]:
num_episodes = 1000
episode_reward = []

for episode in range(num_episodes):
    done = False
    state = env.reset()

    total_reward = 0

    while not done:
        action = 0 #env.action_space.sample()
        state, reward, done, info = env.step(action)

        total_reward += reward

    episode_reward.append(total_reward)

print(f"mean_reward:{np.mean(episode_reward):.2f} +/- {np.std(episode_reward):.2f}")


mean_reward:-1.00 +/- 0.00


In [12]:
num_states = env.NUM_STATES
num_actions = env.NUM_ACTIONS

V = np.zeros(num_states)
optimal_policy = np.zeros([num_states, num_actions])

for s in range(num_states):
	actions_s = np.zeros(num_actions)
	for a in range(num_actions):
		for transition_prob, next_state, reward, done in env.P[s][a]:
			actions_s[a] += reward + transition_prob * V[next_state]
	V[s] = actions_s.max()
	optimal_policy[s, np.argmax(actions_s)] = 1.0


In [25]:
for idx, pol in enumerate(optimal_policy):
    days_left, life_point, field, weather = env.decode(idx)
    print(f"days_left: {days_left}, life_point: {life_point}, field: {field}/{env.NUM_FIELDS}, weather: {env.WEATHER_DICT[weather]}:- action: {env.ACTION_DICT[np.argmax(pol)]}")

days_left: 0, life_point: 0, field: 0/5, weather: Clear:- action: Wait
days_left: 0, life_point: 0, field: 0/5, weather: Rainy:- action: Wait
days_left: 0, life_point: 0, field: 1/5, weather: Clear:- action: Wait
days_left: 0, life_point: 0, field: 1/5, weather: Rainy:- action: Wait
days_left: 0, life_point: 0, field: 2/5, weather: Clear:- action: Wait
days_left: 0, life_point: 0, field: 2/5, weather: Rainy:- action: Wait
days_left: 0, life_point: 0, field: 3/5, weather: Clear:- action: Wait
days_left: 0, life_point: 0, field: 3/5, weather: Rainy:- action: Wait
days_left: 0, life_point: 0, field: 4/5, weather: Clear:- action: Wait
days_left: 0, life_point: 0, field: 4/5, weather: Rainy:- action: Wait
days_left: 0, life_point: 0, field: 5/5, weather: Clear:- action: Wait
days_left: 0, life_point: 0, field: 5/5, weather: Rainy:- action: Wait
days_left: 0, life_point: 1, field: 0/5, weather: Clear:- action: Wait
days_left: 0, life_point: 1, field: 0/5, weather: Rainy:- action: Wait
days_l

In [28]:
for idx, pol in enumerate(optimal_policy):
    days_left, life_point, field, weather = env.decode(idx)
    print(f"days_left: {days_left}, life_point: {life_point}, field: {field}/{env.NUM_FIELDS}, weather: {env.WEATHER_DICT[weather]}:- Value: {V[idx]:.2f}")

days_left: 0, life_point: 0, field: 0/5, weather: Clear:- Value: -1.00
days_left: 0, life_point: 0, field: 0/5, weather: Rainy:- Value: -2.00
days_left: 0, life_point: 0, field: 1/5, weather: Clear:- Value: -2.00
days_left: 0, life_point: 0, field: 1/5, weather: Rainy:- Value: -2.00
days_left: 0, life_point: 0, field: 2/5, weather: Clear:- Value: -2.00
days_left: 0, life_point: 0, field: 2/5, weather: Rainy:- Value: -2.00
days_left: 0, life_point: 0, field: 3/5, weather: Clear:- Value: -2.00
days_left: 0, life_point: 0, field: 3/5, weather: Rainy:- Value: -2.00
days_left: 0, life_point: 0, field: 4/5, weather: Clear:- Value: -2.00
days_left: 0, life_point: 0, field: 4/5, weather: Rainy:- Value: -2.00
days_left: 0, life_point: 0, field: 5/5, weather: Clear:- Value: -2.00
days_left: 0, life_point: 0, field: 5/5, weather: Rainy:- Value: -2.00
days_left: 0, life_point: 1, field: 0/5, weather: Clear:- Value: 0.00
days_left: 0, life_point: 1, field: 0/5, weather: Rainy:- Value: 0.00
days_lef

In [18]:
num_episodes = 2

for episode in range(num_episodes):
    print(f"Episode #{episode+1}")
    print("=" * 10)

    is_dead = False
    obs = env.reset()

    print("Initial state:-")
    env.render()
    print()

    while not is_dead:
        enc_state = env.encode(state["days_left"], state["life_points"], sum(state["field_state"]), state["weather_type"])
        action = np.argmax(optimal_policy[enc_state])
        print(f"Action to take: {env.ACTION_DICT[action]}")
        state, reward, is_dead, info = env.step(action)
        print(f'Chance to find food: {info["chance"]:.2f}')
        print(f"Reward?: {reward}")
        print(f"is Dead?: {is_dead}")
        print()
        print("Current game state:-")
        env.render()
        print()

    print("-" * 10)

Episode #1
Initial state:-
--Days left: 6
--State of Field: [0 0 1 0 1]
--Current life: 5
--Type of Weather: Clear

Action to take: Wait
Chance to find food: 0.00
Reward?: 0
is Dead?: False

Current game state:-
--Days left: 5
--State of Field: [0 1 1 1 1]
--Current life: 4
--Type of Weather: Rainy

Action to take: Forage
Chance to find food: 0.41
Reward?: 0
is Dead?: False

Current game state:-
--Days left: 4
--State of Field: [1 0 1 1 1]
--Current life: 5
--Type of Weather: Rainy

Action to take: Wait
Chance to find food: 0.00
Reward?: 0
is Dead?: False

Current game state:-
--Days left: 3
--State of Field: [1 0 1 0 0]
--Current life: 4
--Type of Weather: Clear

Action to take: Wait
Chance to find food: 0.00
Reward?: 0
is Dead?: False

Current game state:-
--Days left: 2
--State of Field: [1 1 0 0 0]
--Current life: 3
--Type of Weather: Clear

Action to take: Wait
Chance to find food: 0.00
Reward?: 0
is Dead?: False

Current game state:-
--Days left: 1
--State of Field: [1 1 1 0 0]
-

In [19]:
num_episodes = 1000
episode_reward = []

for episode in range(num_episodes):
    done = False
    state = env.reset()

    total_reward = 0

    while not done:
        enc_state = env.encode(state["days_left"], state["life_points"], sum(state["field_state"]), state["weather_type"])
        action = np.argmax(optimal_policy[enc_state])
        state, reward, done, info = env.step(action)

        total_reward += reward

    episode_reward.append(total_reward)

print(f"mean_reward:{np.mean(episode_reward):.2f} +/- {np.std(episode_reward):.2f}")


mean_reward:-0.66 +/- 0.47


In [20]:
model = DQN('MultiInputPolicy', env, exploration_fraction=0.3).learn(total_timesteps=int(2e5))

RuntimeError: Class values must be smaller than num_classes.

In [ ]:
num_episodes = 2

for episode in range(num_episodes):
    print(f"Episode #{episode+1}")
    print("=" * 10)

    is_dead = False
    obs = env.reset()

    print("Initial state:-")
    env.render()
    print()

    while not is_dead:
        action, _ = model.predict(obs)
        print(f"Action to take: {env.ACTION_DICT[action]}")
        state, reward, is_dead, info = env.step(action)
        print(f'Chance to find food: {info["chance"]:.2f}')
        print(f"Reward?: {reward}")
        print(f"is Dead?: {is_dead}")
        print()
        print("Current game state:-")
        env.render()
        print()

    print("-" * 10)

Episode #1
Initial state:-
--Days left: 6
--State of Field: [0 0 1 0 1]
--Current life: 3
--Type of Weather: Rainy



NameError: name 'model' is not defined

In [ ]:
# for idx in range(num_states):
#     field, life_point, weather = env.decode(idx)
#     field_state = np.zeros(env.NUM_FIELDS, dtype=int)
#     field_state[:field] = 1
#     state = {
#         "field_state": field_state,
#         "life_points": life_point,
#         "weather_type": weather
#     }

#     action, _ = model.predict(state, deterministic=True)
#     print(f"field: {field}/{env.NUM_FIELDS}, life_point: {life_point}, weather: {env.WEATHER_DICT[weather]}:- action: {env.ACTION_DICT[action]}")

In [13]:
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=1000)
print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

mean_reward:-2.71 +/- 1.66


In [6]:
self = env

In [ ]:
for days_left in range(self.NUM_DAYS_LEFT):
    for life_point in range(self.NUM_LIFE_POINTS):
        for field in range(self.NUM_FIELDS + 1):
            for weather in range(self.NUM_WEATHER_TYPES):
                prob_success = np.clip(field / self.NUM_FIELDS - weather*self.BAD_WEATHER_EFFECT, 0, 1)
                prob_failure = np.clip(1 - prob_success, 0, 1)

                enc_state = self.encode(days_left, life_point, field, weather)

                for action in range(self.NUM_ACTIONS):
                    if not life_point or not days_left:
                        pass
                    elif action:
                        # forage but fail
                        new_days_left_failure = np.clip(days_left - 1, 0, self.NUM_DAYS_LEFT - 1)
                        new_life_point_failure = np.clip(life_point - 2, 0, self.NUM_LIFE_POINTS - 1)
                        new_field_failure = list(range(self.NUM_FIELDS + 1))
                        new_weather_failure = list(range(self.NUM_WEATHER_TYPES))
                        new_states = list(product(new_days_left_failure, new_life_point_failure, new_field_failure, new_weather_failure))
                        for new_days_left, new_life_point, new_field, new_weather in new_states:
                            if not life_point:
                                self.P[enc_state][action].append([prob_failure / len(new_states), 0, -1, True])
                            elif not days_left:
                                self.P[enc_state][action].append([prob_failure / len(new_states), 0, 1, True])
                            else:
                                new_enc_state = self.encode(new_days_left, new_life_point, new_field, new_weather)
                                self.P[enc_state][action].append([prob_failure / len(new_states), new_enc_state, 0, False])
                        # forage and found
                        new_days_left_success = np.clip(days_left - 1, 0, self.NUM_DAYS_LEFT - 1)
                        new_life_point_success = np.clip(life_point + 1, 0, self.NUM_LIFE_POINTS - 1)
                        new_field_success = list(range(self.NUM_FIELDS + 1))
                        new_weather_success = list(range(self.NUM_WEATHER_TYPES))
                        new_states = list(product(new_days_left_success, new_life_point_success, new_field_success, new_weather_success))
                        for new_days_left, new_life_point, new_field, new_weather in new_states:
                            if not life_point:
                                self.P[enc_state][action].append([prob_success / len(new_states), 0, -1, True])
                            elif not days_left:
                                self.P[enc_state][action].append([prob_success / len(new_states), 0, 1, True])
                            else:
                                new_enc_state = self.encode(new_days_left, new_life_point, new_field, new_weather)
                                self.P[enc_state][action].append([prob_success / len(new_states), new_enc_state, 0, False])
                    else:
                        # wait
                        new_days_left_wait = np.clip(days_left - 1, 0, self.NUM_DAYS_LEFT - 1)
                        new_life_point_wait = np.clip(life_point - 1, 0, self.NUM_LIFE_POINTS - 1)
                        new_field_wait = list(range(self.NUM_FIELDS + 1))
                        new_weather_wait = list(range(self.NUM_WEATHER_TYPES))
                        new_states = list(product(new_days_left_wait, new_life_point_wait, new_field_wait, new_weather_wait))
                        for new_days_left, new_life_point, new_field, new_weather in new_states:
                            if not life_point:
                                self.P[enc_state][action].append([1.0 / len(new_states), 0, -1, True])
                            elif not days_left:
                                self.P[enc_state][action].append([1.0 / len(new_states), 0, 1, True])
                            else:
                                new_enc_state = self.encode(new_days_left, new_life_point, new_field, new_weather)
                                self.P[enc_state][action].append([1.0 / len(new_states), new_enc_state, 0, False])

In [7]:
for days_left in range(self.NUM_DAYS_LEFT + 1):
    for life_point in range(self.NUM_LIFE_POINTS):
        for field in range(self.NUM_FIELDS + 1):
            for weather in range(self.NUM_WEATHER_TYPES):
                prob_success = np.clip(
                    field / self.NUM_FIELDS - weather*self.BAD_WEATHER_EFFECT, 0, 1
                )
                prob_failure = np.clip(
                    1 - prob_success, 0, 1
                )
                enc_state = self.encode(days_left, life_point, field, weather)
                for action in range(self.NUM_ACTIONS):
                    if not life_point:
                        self.P[enc_state][action].append([1, 0, -1, True])
                    elif not days_left:
                        self.P[enc_state][action].append([1, 0, 1, True])
                    elif life_point and action:
                        new_life_point_failure = np.clip(
                            life_point - 2, 0, self.NUM_LIFE_POINTS - 1
                        )
                        new_days_left_failure = np.clip(
                            days_left - 1, 0, self.NUM_DAYS_LEFT - 1
                        )
                        new_enc_state = self.encode(
                            new_days_left_failure, new_life_point_failure, field, weather
                        )
                        if new_life_point_failure:
                            self.P[enc_state][action].append(
                                [prob_failure, new_enc_state, 0, False]
                            )
                        else:
                            self.P[enc_state][action].append(
                                [prob_failure, 0, -1, True]
                            )
                        new_life_point_success = np.clip(
                            life_point + 1, 0, self.NUM_LIFE_POINTS - 1
                        )
                        new_days_left_success = np.clip(
                            days_left - 1, 0, self.NUM_DAYS_LEFT - 1
                        )
                        new_enc_state = self.encode(
                            new_days_left_success, new_life_point_success, field, weather
                        )
                        if new_life_point_success:
                            self.P[enc_state][action].append(
                                [prob_success, new_enc_state, 0, False]
                            )
                        else:
                            self.P[enc_state][action].append(
                                [prob_success, 0, -1, True]
                            )
                    else:
                        new_life_point_wait = np.clip(
                            life_point - 1, 0, self.NUM_LIFE_POINTS - 1
                        )
                        new_days_left_wait = np.clip(
                            days_left - 1, 0, self.NUM_DAYS_LEFT - 1
                        )
                        new_enc_state = self.encode(
                            new_days_left_wait, new_life_point_wait, field, weather
                        )
                        if new_life_point_wait:
                            self.P[enc_state][action].append(
                                [1.0, new_enc_state, 0, False]
                            )
                        else:
                            self.P[enc_state][action].append(
                                [1.0, 0, -1, True]
                            )

In [22]:
a = [np.clip(days_left - 1, 0, self.NUM_DAYS_LEFT - 1)]
a

[5]

In [23]:
b = [np.clip(life_point - 2, 0, self.NUM_LIFE_POINTS - 1)]
b

[4]

In [24]:
c = list(range(self.NUM_FIELDS + 1))
c

[0, 1, 2, 3, 4, 5]

In [25]:
d = list(range(self.NUM_WEATHER_TYPES))
d

[0, 1]

In [28]:
list(product(a, b, c, d))

[(5, 4, 0, 0),
 (5, 4, 0, 1),
 (5, 4, 1, 0),
 (5, 4, 1, 1),
 (5, 4, 2, 0),
 (5, 4, 2, 1),
 (5, 4, 3, 0),
 (5, 4, 3, 1),
 (5, 4, 4, 0),
 (5, 4, 4, 1),
 (5, 4, 5, 0),
 (5, 4, 5, 1)]